In [1]:
import os

In [3]:
os.chdir('../')

In [8]:
%pwd

'c:\\Users\\HAPPYUSER\\Documents\\Web_Development\\ML_AI_MLOPS\\Sports-image-classification'

In [20]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_folder: Path
    validation_folder: Path
    params_epochs: int
    params_batch_size: int
    params_image_size: list
    

In [21]:
from SportsImageClassifier.constants import *
from SportsImageClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [22]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_folder = os.path.join(self.config.data_ingestion.unzip_dir, "train")
        validation_folder = os.path.join(self.config.data_ingestion.unzip_dir, "valid")
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_folder=Path(training_folder),
            validation_folder=Path(validation_folder),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            # params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [24]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_data(self):

        train_data = tf.keras.preprocessing.image_dataset_from_directory(
            self.config.training_folder, 
            image_size=self.config.params_image_size[:-1], 
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
            )
        validation_data = tf.keras.preprocessing.image_dataset_from_directory(
            self.config.validation_folder,
            image_size=self.config.params_image_size[:-1], 
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
            )
        self.scaled_train_data = train_data.map(lambda x,y:(x/255,y)) 
        self.scaled_val_data = validation_data.map(lambda x,y: (x/255, y))

        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



    
    def train(self):
        
        self.model.fit(
            self.scaled_train_data,
            epochs=self.config.params_epochs,
            # steps_per_epoch=self.steps_per_epoch,
            # validation_steps=self.validation_steps,
            validation_data=self.scaled_val_data
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [26]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_data()
    training.train()
    
except Exception as e:
    raise e

[2024-09-02 23:12:51,635: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-09-02 23:12:51,641: INFO: common: yaml file: params.yaml loaded successfully]
[2024-09-02 23:12:51,644: INFO: common: created directory at: artifacts]
[2024-09-02 23:12:51,647: INFO: common: created directory at: artifacts\training]


c:\Users\HAPPYUSER\Documents\Web_Development\ML_AI_MLOPS\Sports-image-classification\venv\lib\site-packages\keras\src\saving\saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 13492 files belonging to 100 classes.
Found 500 files belonging to 100 classes.
Epoch 1/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 477s 1s/step - accuracy: 0.0095 - loss: 4.8746 - val_accuracy: 0.0100 - val_loss: 4.6230
Epoch 2/2
422/422 ━━━━━━━━━━━━━━━━━━━━ 496s 1s/step - accuracy: 0.0110 - loss: 4.7275 - val_accuracy: 0.0100 - val_loss: 4.6122
